Now that we've seen the story that the data tells, it's time to build the model. First we import the libraries and the data file. 

In [34]:
%matplotlib inline
import numpy as np
import scipy as sp
import matplotlib as mpl
import matplotlib.cm as cm
from matplotlib.colors import ListedColormap
import matplotlib.pyplot as plt
import pandas as pd
pd.set_option('display.width', 500)
pd.set_option('display.max_columns', 100)
pd.set_option('display.notebook_repr_html', True)
import seaborn as sns
sns.set_style("whitegrid")
sns.set_context("poster")
import sklearn.model_selection

c0=sns.color_palette()[0]
c1=sns.color_palette()[1]
c2=sns.color_palette()[2]

cmap_light = ListedColormap(['#FFAAAA', '#AAFFAA', '#AAAAFF'])
cmap_bold = ListedColormap(['#FF0000', '#00FF00', '#0000FF'])
cm = plt.cm.RdBu
cm_bright = ListedColormap(['#FF0000', '#0000FF'])

def points_plot(ax, Xtr, Xte, ytr, yte, clf, mesh=True, colorscale=cmap_light, 
                cdiscrete=cmap_bold, alpha=0.1, psize=10, zfunc=False, predicted=False):
    h = .02
    X=np.concatenate((Xtr, Xte))
    x_min, x_max = X[:, 0].min() - .5, X[:, 0].max() + .5
    y_min, y_max = X[:, 1].min() - .5, X[:, 1].max() + .5
    xx, yy = np.meshgrid(np.linspace(x_min, x_max, 100),
                         np.linspace(y_min, y_max, 100))

    #plt.figure(figsize=(10,6))
    if zfunc:
        p0 = clf.predict_proba(np.c_[xx.ravel(), yy.ravel()])[:, 0]
        p1 = clf.predict_proba(np.c_[xx.ravel(), yy.ravel()])[:, 1]
        Z=zfunc(p0, p1)
    else:
        Z = clf.predict(np.c_[xx.ravel(), yy.ravel()])
    ZZ = Z.reshape(xx.shape)
    if mesh:
        plt.pcolormesh(xx, yy, ZZ, cmap=cmap_light, alpha=alpha, axes=ax)
    if predicted:
        showtr = clf.predict(Xtr)
        showte = clf.predict(Xte)
    else:
        showtr = ytr
        showte = yte
    ax.scatter(Xtr[:, 0], Xtr[:, 1], c=showtr-1, cmap=cmap_bold, 
               s=psize, alpha=alpha,edgecolor="k")
    # and testing points
    ax.scatter(Xte[:, 0], Xte[:, 1], c=showte-1, cmap=cmap_bold, 
               alpha=alpha, marker="s", s=psize+10)
    ax.set_xlim(xx.min(), xx.max())
    ax.set_ylim(yy.min(), yy.max())
    return ax,xx,yy

def points_plot_prob(ax, Xtr, Xte, ytr, yte, clf, colorscale=cmap_light, 
                     cdiscrete=cmap_bold, ccolor=cm, psize=10, alpha=0.1):
    ax,xx,yy = points_plot(ax, Xtr, Xte, ytr, yte, clf, mesh=False, 
                           colorscale=colorscale, cdiscrete=cdiscrete, 
                           psize=psize, alpha=alpha, predicted=True) 
    Z = clf.predict_proba(np.c_[xx.ravel(), yy.ravel()])[:, 1]
    Z = Z.reshape(xx.shape)
    plt.contourf(xx, yy, Z, cmap=ccolor, alpha=.2, axes=ax)
    cs2 = plt.contour(xx, yy, Z, cmap=ccolor, alpha=.6, axes=ax)
    plt.clabel(cs2, fmt = '%2.1f', colors = 'k', fontsize=14, axes=ax)
    return ax 

print("Loaded")

Loaded


In [42]:
df = pd.read_csv (r"C:\Users\Laura-Black\Documents\PhD\Data Scientist\Springboard\Capstone Projects\Drug Abuse ED Visits\Output files\Capstone1DataWrangling-NOLabels.csv", 
                     index_col=0, low_memory=False)

print(df.head())
print(df.info())

        METRO  STRATA  PSU  REPLICATE   CASEWGT  PSUFRAME  AGECAT  SEX  RACE  YEAR  QUARTER  DAYPART  NUMSUBS  DRUGID_1  ROUTE_1  TOXTEST_1  DRUGID_2  ROUTE_2  TOXTEST_2  DRUGID_3  ROUTE_3  TOXTEST_3  DRUGID_4  ROUTE_4  TOXTEST_4  DRUGID_5  ROUTE_5  TOXTEST_5  DRUGID_6  ROUTE_6  TOXTEST_6  DRUGID_7  ROUTE_7  TOXTEST_7  DRUGID_8  ROUTE_8  TOXTEST_8  DRUGID_9  ROUTE_9  TOXTEST_9  DRUGID_10  ROUTE_10  TOXTEST_10  DRUGID_11  ROUTE_11  TOXTEST_11  DRUGID_12  ROUTE_12  TOXTEST_12  DRUGID_13  \
CASEID                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      
1           2 

Next, we will perform a logistic regression analysis on CASETYPE vs METRO, AGECAT, SEX and RACE. We will start by defining the x and y variables.

In [73]:
feature_columns = ['METRO', 'AGECAT', 'SEX', 'RACE']
X = df[feature_columns]
print(x)
y = df['CASETYPE']
print('\n',y)


              METRO AGECAT     SEX      RACE ALCOHOL              PHARMA
CASEID                                                                  
1          New York  18-20    Male     Black     Yes  No Pharmaceuticals
2          New York   > 65    Male  Hispanic      No     Pharmaceuticals
3       Date County   > 65  Female     Black      No     Pharmaceuticals
4           Phoenix  06-11    Male  Hispanic      No     Pharmaceuticals
5            Boston  25-29    Male  Hispanic     Yes  No Pharmaceuticals
...             ...    ...     ...       ...     ...                 ...
229207  Minneapolis   > 65  Female     White      No     Pharmaceuticals
229208      Seattle  45-54  Female     White      No     Pharmaceuticals
229209  Minneapolis  35-44  Female   Not Doc      No     Pharmaceuticals
229210      Chicago    0-5  Female     Black      No     Pharmaceuticals
229211      Detroit  55-64  Female     Black      No     Pharmaceuticals

[229211 rows x 6 columns]

 CASEID
1         8
2  

In [58]:
#Split the data into a training and test set.
Xlr, Xtestlr, ylr, ytestlr = train_test_split(X, y, random_state=5)

In [59]:
print("\n")
print("Xlr:", Xlr, type(Xlr), Xlr.shape, len(Xlr)) #TrainX


print("\n")
print("Xtestlr", Xtestlr, type(Xtestlr), Xtestlr.shape, len(Xtestlr)) #TestX


print("\n")
print("ylr", ylr, type(ylr), ylr.shape, len(ylr)) #Trainy


print("\n")
print("ytestlr", ytestlr, type(ytestlr), ytestlr.shape, len(ytestlr)) #Testy



Xlr:         METRO  AGECAT  SEX  RACE  ALCOHOL  PHARMA
CASEID                                           
110518      3       9    1    -8        0       0
113807      2       9    1     1        0       1
14696       3       8    1     1        0       0
6142        2       8    1     3        1       0
61549       4       9    2     1        0       1
...       ...     ...  ...   ...      ...     ...
121975      9       7    2     1        1       1
124606      1       6    2    -8        0       1
20464       2       9    1    -8        0       1
18639       2       8    1     2        1       0
35684       9      11    2     3        0       1

[171908 rows x 6 columns] <class 'pandas.core.frame.DataFrame'> (171908, 6) 171908


Xtestlr         METRO  AGECAT  SEX  RACE  ALCOHOL  PHARMA
CASEID                                           
12784       4       7    1     1        1       0
210708     14       8    2     1        0       1
103453      3      10    2     1        0       1

In [74]:
# construct the LogisticRegression model
clf = LogisticRegression(solver='lbfgs', multi_class='auto', max_iter=10000)

# Fit the model on the training data.
clf.fit(Xlr, ylr) 

# Print the accuracy from the testing data.
# introduce variable to be reused later
y_predict_test = clf.predict(Xtestlr)
print("\n")
print("[Test] Accuracy score (y_predict_test, ytestlr):",accuracy_score(y_predict_test, ytestlr))

print("\n")
print("[Test] Accuracy score: (ytestlr, y_predict_test)",accuracy_score(ytestlr, y_predict_test))

# also printout the training score
y_predict_training = clf.predict(Xlr)
print("\n")
print("[Training] Accuracy score: (ylr, y_predict_training)",accuracy_score(ylr, y_predict_training))



[Test] Accuracy score (y_predict_test, ytestlr): 0.6738041638308641


[Test] Accuracy score: (ytestlr, y_predict_test) 0.6738041638308641


[Training] Accuracy score: (ylr, y_predict_training) 0.6698117597784862


In [76]:
# import the metrics class
from sklearn import metrics
cnf_matrix = metrics.confusion_matrix(ytestlr, y_predict_test)
cnf_matrix

array([[    0,     0,    18,  1471,     0,     0,     1,   741],
       [    0,     0,    21,   988,     0,     0,     1,  2699],
       [    0,     0,  1844,     0,     0,     0,     0,     1],
       [    0,     0,     0, 21684,     0,     0,     5,   357],
       [    0,     0,     0,  3800,     0,     0,     1,   711],
       [    0,     0,     9,    61,     0,     0,     0,   140],
       [    0,     0,     0,   778,     0,     0,     0,    27],
       [    0,     1,   475,  6382,     0,     0,     4, 15083]],
      dtype=int64)

In [77]:
#calculate the F score
from sklearn.metrics import classification_report
print(classification_report(ytestlr, y_predict_test))

              precision    recall  f1-score   support

           1       0.00      0.00      0.00      2231
           2       0.00      0.00      0.00      3709
           3       0.78      1.00      0.88      1845
           4       0.62      0.98      0.76     22046
           5       0.00      0.00      0.00      4512
           6       0.00      0.00      0.00       210
           7       0.00      0.00      0.00       805
           8       0.76      0.69      0.72     21945

    accuracy                           0.67     57303
   macro avg       0.27      0.33      0.29     57303
weighted avg       0.55      0.67      0.60     57303



C:\Users\Laura-Black\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
